<a href="https://colab.research.google.com/github/Regent87/Neuro_Spase/blob/main/Demo_NeuroSpase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Выполните ячейку что бы загрузить файлы с сеансами видимости спутник станция**

In [ ]:
from google.colab import files
#Загрузка файлов с сеансами видимости спутник-станция
#Обязательно дождитесь конца загрузки
uploaded = files.upload()


**Выполнити ячейку что бы загрузить файлы с сеансами пересечения границы России**

In [ ]:
#Загрузка файлов с сеансами пересечения границ России
uploaded2 = files.upload()

# Запустите ячейку что бы выполнить программу

## Зазружаем библиотеки и функции

### Загружаем библиотеки

In [6]:
from pprint import pprint
import pandas as pd
import datetime
import math
import os

### Функция переписывает файлы в папке txt в csv файлы

In [7]:
def txt_to_csv(way, mod=0): # mod= 0 для файлов границ россии
                            # mod = 1 для файлов с окнами связи спутник-станция
  # На вход принимает путь к txt файлу.
  t1 = datetime.time(9,0,0,0)
  t2 = datetime.time(18,0,0,0)
  sl_data = []
  data = []
  data1 = []
  name = []
  name1 = []
  access = []
  start_time = []
  stop_time = []
  duration = []
  duration_bit = []
  with open(way) as f:
    m = f.readlines()
    for i in range(3, len(m)):
      if '-To-' in m[i]:
        sl_data.append(m[i])
        st_sp = m[i][:-1].split('-To-')
        if st_sp[0] not in data:
          data.append(st_sp[0])
        if st_sp[1][-6:] not in data1:
          data1.append(int(st_sp[1][-6:]))

    for i in range(len(sl_data)-1):
      st_sp = sl_data[i][:-1].split('-To-')
      for j in range(m.index(sl_data[i])+4, m.index(sl_data[i+1])-7):
        name.append(st_sp[0])
        name1.append(int(st_sp[1][-6:]))
        access.append(int(m[j][:24]))
        t_s0 = datetime.datetime.strptime(m[j][28:52], '%d %b %Y %H:%M:%S.%f')
        t_s1 = datetime.datetime.strptime(m[j][56:80], '%d %b %Y %H:%M:%S.%f')
        time1 = datetime.datetime.combine(t_s0.date(), t1)
        time2 = datetime.datetime.combine(t_s0.date(), t2)

        start_time.append(t_s0)
        stop_time.append(t_s1)
        duration.append(float(m[j][81:]))

        if mod == 0:
          duration_bit.append(float(m[j][81:])*4)
        else:
          if int(st_sp[1][-6:]) < 110600:  
            duration_bit.append(float(m[j][81:]))
          else:
            duration_bit.append(float(m[j][81:])*0.25)

    i = len(sl_data)-1

    for k in range(m.index(sl_data[i])+4, len(m)-12):
      st_sp = sl_data[i][:-1].split('-To-')
      name.append(st_sp[0])
      name1.append(int(st_sp[1][-6:]))
      access.append(int(m[k][:24]))
      t_s0 = datetime.datetime.strptime(m[k][28:52], '%d %b %Y %H:%M:%S.%f')
      t_s1 = datetime.datetime.strptime(m[k][56:80], '%d %b %Y %H:%M:%S.%f')
      time1 = datetime.datetime.combine(t_s0.date(), t1)
      time2 = datetime.datetime.combine(t_s0.date(), t2)
      start_time.append(t_s0)
      stop_time.append(t_s1)
      duration.append(float(m[k][81:]))

      if mod == 0:
        duration_bit.append(float(m[k][81:])*4)


      else:
        if int(st_sp[1][-6:]) < 110600:
          duration_bit.append(float(m[j][81:]))
        else:
          duration_bit.append(float(m[j][81:])*0.25)

  data_fr = {'Name_station': name, 'Name_sat': name1, 'Access': access, 'Start Time':  start_time, 'Stop Time': stop_time, 'Duration': duration, 'Duration_bit': duration_bit}
  # возвращает датафрейм
  if mod == 0:
    return pd.DataFrame(data_fr), data, data1
  else:
    return pd.DataFrame(data_fr), data





### Функция генерирует датафрейм с рассписанием

In [8]:
def Generator_timing(df_res, st, sp): # На вход принимает датафрейм с данными, список станций, список спутников
  data_zirro_m = {'Access': [],
                  'Start Time (UTCG)': [],
                  'Stop Time (UTCG)': [],
                  'Duration (sec)': [],
                  'Sat name': [],
                  'Data (Mbytes)': [],
                  'Name_station': [],
                  'Give Data':[],
                  'Get Data':[]
                  }

  connect_dict = {}
  for i in st:
    connect_dict[i] = {'Connect':{}, 
                      'Vis_sat': {}}

  sat_status_dict ={}
  for i in sp:
    sat_status_dict[i]= Satellite(i, 0 , 0, 0, 0, 0, 0)


  stop_time_sp = []
  station_sp1 = []
  station_sp = st.copy()
  t_start = min(df_res.loc[df_res['Name_station']=='Russia']['Start Time'])
  t_day_start = 0
  t_day_stop = (datetime.datetime(2027,6,14,0,0,0,0) - 
                datetime.datetime(2027,6,1,0,0,0,0)).total_seconds()
  t_delta = 83.5
  flag = 0


  while t_start <= t_day_stop:
    t_stop = t_start +  t_delta
    df_r1 = df_res.loc[(df_res['Start Time']>=t_start)&(df_res['Start Time']<=t_stop)]
    for i in station_sp:
      df_r = df_r1.loc[(df_r1['Name_station']==i)]
      if i == 'Russia':
        station_sp1.append(i)
        for k in df_r['Name_sat']:
          index_s = df_r.index[df_r['Name_sat'] == k].tolist()[0]
          if (k not in connect_dict[i]['Connect'] and 
              sat_status_dict[k].connect == 0 and 
              sat_status_dict[k].data_now + df_r['Duration_bit'][index_s]<(8192, 4096)[k>110600]):
            sat_status_dict[k].connect = i
            sat_status_dict[k].start_time = df_r['Start Time'][index_s]
            sat_status_dict[k].stop_time = df_r['Stop Time'][index_s]
            sat_status_dict[k].data_now += df_r['Duration_bit'][index_s]
            sat_status_dict[k].data_get = df_r['Duration_bit'][index_s]
            sat_status_dict[k].data_give = 0
            stop_time_sp.append([sat_status_dict[k].stop_time, k, i])
            connect_dict[i]['Connect'][k] = sat_status_dict[k]
        
        
      if i != 'Russia':
        sat_sp = []
        for k in df_r['Name_sat']:
          index_s = df_r.index[df_r['Name_sat'] == k].tolist()[0]
          if (k not in connect_dict[i]['Vis_sat'] and
              sat_status_dict[k].connect == 0 and 
              sat_status_dict[k].data_now > 0):
            data_give = (df_r['Duration_bit'][index_s], 
                        sat_status_dict[k].data_now
                        )[df_r['Duration_bit'][index_s]>
                          sat_status_dict[k].data_now]

            if df_r['Duration_bit'][index_s] > sat_status_dict[k].data_now:
              data_give = sat_status_dict[k].data_now
              stop_time1 = df_r['Start Time'][index_s] + data_give/(1, 0.25)[k > 110600]
            else:
              data_give = df_r['Duration_bit'][index_s]
              stop_time1 = df_r['Stop Time'][index_s]
            connect_dict[i]['Vis_sat'][k] = Data_status(k, 
                                                        data_give, 
                                                        0, 
                                                        df_r['Start Time'][index_s], 
                                                        stop_time1)
            sat_sp.append(connect_dict[i]['Vis_sat'][k])
        sat_sp.sort(key = lambda x : x.data_give)
        if len(sat_sp)==0:
          station_sp1.append(i)
        if len(sat_sp)>0:
          item = sat_sp.pop()
          k1 = item.name

          sat_status_dict[k1].connect = i
          sat_status_dict[k1].start_time = connect_dict[i]['Vis_sat'][k1].start_time
          sat_status_dict[k1].stop_time = connect_dict[i]['Vis_sat'][k1].stop_time
          sat_status_dict[k1].data_now -= connect_dict[i]['Vis_sat'][k1].data_give
          sat_status_dict[k1].data_get = 0
          sat_status_dict[k1].data_give = connect_dict[i]['Vis_sat'][k1].data_give
          stop_time_sp.append([sat_status_dict[k1].stop_time, k1, i])
          connect_dict[i]['Connect'][k1] = sat_status_dict[k1]
          if (len(sat_sp)>0 and 
              sat_status_dict[k1].start_time > t_start and
              min(sat_sp, key = lambda x : x.start_time).start_time < item.start_time):
            sat_sp.sort(key = lambda x : (-x.start_time))

            item2 = sat_sp.pop()
            
            l = item2.name
      
            sat_status_dict[l].connect = i
            sat_status_dict[l].start_time = connect_dict[i]['Vis_sat'][l].start_time
            
            m = (1, 0.25)[l > 110600]
            if connect_dict[i]['Vis_sat'][l].data_give <= (connect_dict[i]['Vis_sat'][k1].start_time - 
                                                          sat_status_dict[l].start_time)*m:
              sat_status_dict[l].stop_time = connect_dict[i]['Vis_sat'][l].stop_time
              sat_status_dict[l].data_give = connect_dict[i]['Vis_sat'][l].data_give
              sat_status_dict[l].data_now -= connect_dict[i]['Vis_sat'][l].data_give
    
            else:
              sat_status_dict[l].stop_time = connect_dict[i]['Vis_sat'][k1].start_time
              sat_status_dict[l].data_give = (sat_status_dict[l].stop_time - 
                                              sat_status_dict[l].start_time)*m
              sat_status_dict[l].data_now -= sat_status_dict[l].data_give
            sat_status_dict[l].data_get = 0
            connect_dict[i]['Connect'][l] = sat_status_dict[l]
            stop_time_sp.append([sat_status_dict[l].stop_time, l, i])

    stop_time_sp.sort(key= lambda x: -x[0])
    if len(stop_time_sp) != 0:
      item_t = stop_time_sp.pop()
      data_zirro_m['Access'].append(flag)
      data_zirro_m['Start Time (UTCG)'].append(datetime.datetime(2027,6,1,0,0,0,0) + datetime.timedelta(seconds=sat_status_dict[item_t[1]].start_time))
      data_zirro_m['Stop Time (UTCG)'].append(datetime.datetime(2027,6,1,0,0,0,0) + datetime.timedelta(seconds=item_t[0]))
      data_zirro_m['Duration (sec)'].append(item_t[0] - sat_status_dict[item_t[1]].start_time)
      data_zirro_m['Sat name'].append(item_t[1])
      data_zirro_m['Data (Mbytes)'].append(sat_status_dict[item_t[1]].data_now)
      data_zirro_m['Name_station'].append(item_t[2])
      data_zirro_m['Give Data'].append(sat_status_dict[item_t[1]].data_give)
      data_zirro_m['Get Data'].append(sat_status_dict[item_t[1]].data_get)

      sat_status_dict[item_t[1]].connect = 0
      sat_status_dict[item_t[1]].start_time = 0
      sat_status_dict[item_t[1]].stop_time = 0
      sat_status_dict[item_t[1]].data_give = 0
      sat_status_dict[item_t[1]].data_get = 0

      flag+=1
      if (item_t[2] != 'Russia' and 
          len(connect_dict[item_t[2]]['Connect'])==1):
        station_sp1.append(item_t[2])

      del connect_dict[item_t[2]]['Connect'][item_t[1]]
      if item_t[2] != 'Russia':
        del connect_dict[item_t[2]]['Vis_sat'][item_t[1]]



      station_sp = station_sp1.copy()
      t_start = item_t[0]
      #print(len(station_sp1))
      station_sp1 = []
    
    else:
      station_sp = station_sp1.copy()
      t_start += t_delta
      #print(len(station_sp1))
      station_sp1 = []
  return pd.DataFrame(data_zirro_m) # возвращает датафорейм с рассписанием


### Функция генерирует txt файлы из датафрейма

In [20]:
def csv2txt(d_frame, path, st): # на входд принимает: 
  station_sp = []
  for station in st:            # датафрейм с рассчетами рассписания
    if station != 'Russia':     # путь куда сохранять файлы и список станций
      station_sp.append(f'{station}.txt')
      with open(f'{path}{station}.txt', "w") as file1:
        file1.write(f'{station}\n')
        file1.write('-------------------------\n')
        file1.write('Access        Start Time (UTCG)           Stop Time (UTCG)        Duration (sec)        Sat name        Data (Mbytes)\n')
        file1.write('------        -----------------           ----------------        --------------        --------        -------------\n')
        
        df_writ = frame.loc[frame['Name_station'] == station]
        for i in range(len(df_writ)):
          
          sat_name1 = str(df_writ['Sat name'].tolist()[i])
          sat_name = f'KinoSat_{sat_name1}'

          stop_data = df_writ['Stop Time (UTCG)'].tolist()[i].strftime("%d %b %Y %H:%M:%S.%f")[:-3]
          start_data = df_writ['Start Time (UTCG)'].tolist()[i].strftime("%d %b %Y %H:%M:%S.%f")[:-3]
          if stop_data[0]=='0':
            stop_data = ' '+stop_data[1:]
          if start_data[0]=='0':
            start_data = ' '+start_data[1:]

          duration = ' '*14
          duration1 = str(round(df_writ['Duration (sec)'].tolist()[i],3))
          duration = duration[:-len(duration1)]+duration1

          data_mbytes = ' '*13
          data_mbytes1 = str(round(df_writ['Data (Mbytes)'].tolist()[i],3))
          data_mbytes = data_mbytes[:-len(data_mbytes1)]+data_mbytes1

          access = ' '*6
          access1 = str(i+1)
          access = access[:-len(access1)] + access1

          file1.write(f'{access}     {start_data}    {stop_data}   {duration}      {sat_name}    {data_mbytes}\n')
  return station_sp

### Функция генерирует 1 большой TXT со всеми данными

In [10]:
def csv2BigFile(path, frame):
  with open(f'{path}/result.txt', "w") as file1:

    file1.write('Access   Start Time (UTCG)        Stop Time (UTCG)        Duration (sec)     Sat name    Data (Mbytes)   Station    Give Data     Get Data\n')
    file1.write('------   -----------------        ----------------        --------------     --------    -------------   -------    ---------     --------\n')


    for i in range(len(frame)):
      sat_name1 = str(frame['Sat name'].tolist()[i])
      sat_name = f'KinoSat_{sat_name1}'

      stop_data = frame['Stop Time (UTCG)'].tolist()[i].strftime("%d %b %Y %H:%M:%S.%f")[:-3]
      start_data = frame['Start Time (UTCG)'].tolist()[i].strftime("%d %b %Y %H:%M:%S.%f")[:-3]
      if stop_data[0]=='0':
        stop_data = ' '+stop_data[1:]
      if start_data[0]=='0':
        start_data = ' '+start_data[1:]

      duration = ' '*14
      duration1 = str(round(frame['Duration (sec)'].tolist()[i],3))
      duration = duration[:-len(duration1)]+duration1

      data_mbytes = ' '*12
      data_mbytes1 = str(round(frame['Data (Mbytes)'].tolist()[i],3))
      data_mbytes = data_mbytes[:-len(data_mbytes1)]+data_mbytes1

      access = ' '*6
      access1 = str(i+1)
      access = access[:-len(access1)] + access1

      station = ' '*13
      station1 = frame['Name_station'].tolist()[i]
      station = station[: -len(station1)] + station1

      station = ' '*13
      station1 = frame['Name_station'].tolist()[i]
      station = station[: -len(station1)] + station1

      give = ' '*10
      give1 = str(round(frame['Give Data'].tolist()[i],3))
      give = give[: -len(give1)] + give1

      get_d = ' '*10
      get_d1 = str(round(frame['Get Data'].tolist()[i],3))
      get_d = get_d[: -len(get_d1)] + get_d1

      sum_giv = ' '*14
      sum_giv1 = str(round(sum(frame['Give Data']),3))
      sum_giv = sum_giv[: -len(sum_giv1)] + sum_giv1

      sum_get = ' '*14
      sum_get1 = str(round(sum(frame['Get Data']),3))
      sum_get = sum_get[: -len(sum_get1)] + sum_get1

      sum_dur = ' '*14
      sum_dur1 = str(round(sum(frame['Get Data']),3))
      sum_dur = sum_dur[: -len(sum_dur1)] + sum_dur1

      file1.write(f'{access} {start_data} {stop_data}{duration}   {sat_name}{data_mbytes}{station}  {give}    {get_d}\n')

    file1.write('----------------------\n')  
    file1.write(f'TOTAL                                                    {sum_dur}                                      {sum_giv} {sum_get}\n')
      

## Начинаем выполнять программу

### Создаем 2 новых новый класса:
### спутник и видымый спутник


In [11]:
class Satellite:
  def __init__(self, name, data_now, data_give, data_get, connect, start_time, stop_time):
    self.name = name
    self.data_now = data_now
    self.data_give = data_give
    self.data_get = data_get
    self.connect = connect
    self.start_time = start_time
    self.stop_time = stop_time
  def __repr__(self):
    return repr((self.name, 
                 self.data_now, 
                 self.data_give, 
                 self.data_get, 
                 self.connect,
                 self.start_time,
                 self.stop_time))
class Data_status:
  def __init__(self, name, data_give, data_get, start_time, stop_time):
    self.name = name
    self.data_give = data_give
    self.data_get = data_get
    self.start_time = start_time
    self.stop_time = stop_time
  def __repr__(self):
    return repr((self.name, 
                 self.data_give, 
                 self.data_get, 
                 self.start_time,
                 self.stop_time))

### Создаем датафрейм

In [13]:
Rus = []
fac = []
st, sp = [], []
for i in uploaded2:
  Rus.append(i)
for i in uploaded:
  fac.append(i)

df1, st, sp = txt_to_csv(Rus[0])
for i in range(1, len(Rus)):
  df2, st2, sp2 = txt_to_csv(Rus[i])
  if st2[0] not in st:
    st.append(st2[0])
  sp += sp2
  df1 = df1.append(df2, ignore_index=True)

df11, st1 = txt_to_csv(fac[0],1)
st += st1
for i in range(1, len(fac)):
  df22, st11 = txt_to_csv(fac[i],1)
  if st11[0] not in st:
    st.append(st11[0])
  df11 = df11.append(df22, ignore_index=True)

#создаем результирующий список
df_res= df11.append(df1, ignore_index=True)
# сохраняем результирующий датафрейм
df_res.to_csv('Russia2Facility2Constellation.csv')

#сортируем список
df_res.sort_values(['Start Time', 'Stop Time'], inplace=True, ignore_index=True)
#переводим время в секунды
t_s0 = datetime.datetime(2027,6,1,0,0,0,0)
df_res['Start Time'] = df_res['Start Time'].apply(lambda x: (x - t_s0).total_seconds())
df_res['Stop Time'] = df_res['Stop Time'].apply(lambda x: (x - t_s0).total_seconds())

<ipython-input-13-fa6a1810e297>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append(df2, ignore_index=True)
<ipython-input-13-fa6a1810e297>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df11 = df11.append(df22, ignore_index=True)
<ipython-input-13-fa6a1810e297>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df11 = df11.append(df22, ignore_index=True)
<ipython-input-13-fa6a1810e297>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df11 = df11.append(df22, ignore_index=True)
<ipython-input-13-fa6a1810e297>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

### Генерируем расписание

In [14]:
frame = Generator_timing(df_res, st, sp)

<ipython-input-8-ee45a79c9979>:63: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  data_give = (df_r['Duration_bit'][index_s],


In [16]:
d_Kino = sum((frame.loc[(frame['Name_station'] != 'Russia') & 
 (frame['Sat name'] < 110600)]['Duration (sec)']))*1/8192

d_Zor = sum((frame.loc[(frame['Name_station'] != 'Russia') & 
 (frame['Sat name'] > 110600)]['Duration (sec)']))*0.25/8192
d_Rus = sum((frame.loc[(frame['Name_station'] == 'Russia')]['Duration (sec)']))*4/8192
print(f'Kinosat передали {round(d_Kino, 3)}ТБ')
print(f'Zorkii передали {round(d_Zor, 3)}ТБ')
print(f'Все спутники передали {round(d_Zor, 3) + round(d_Kino, 3)}ТБ')
print(f'все спутники приняли {round(d_Rus, 3)}ТБ')

Kinosat передали 333.281ТБ
Zorkii передали 259.22ТБ
Все спутники передали 592.501ТБ
все спутники приняли 654.172ТБ


### Сохраняем рассписание в TXT 

In [21]:
#айлы начнут автоматически скачиваться на компьютер
path = ''
station_sp = csv2txt(frame, path, st)
for i in station_sp: 
  files.download(i)